In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as  tf
import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.optimizers import RMSprop
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers


import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw
import random
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
test= pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')

In [ ]:
train_data = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test_data =pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
sample = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')
pd.set_option('display.max_columns', None)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.info()


In [ ]:
test_data.info()

In [ ]:
sample.head()

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data.corr()

In [ ]:
train_data['target'].unique()

In [ ]:
labels=['Class_1', 'Class_2', 'Class_3', 'Class_4']
colors=['gold', 'yellowgreen', 'lightcoral', 'lightskyblue']
explode = (0.1, 0, 0, 0)

plt.figure(figsize=(8,8))
plt.pie(train_data['target'].value_counts(), explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=True, startangle=140)

plt.axis('equal')
plt.show()

In [ ]:
train_data.describe()

# **Normalize the data**

In [ ]:
test_data= test_data.drop(['id'], axis=1)
train_data= train_data.drop(['id'], axis=1)
train= train_data.drop(['target'], axis=1)
num_features = train.shape[1]
num_features

In [ ]:
for i in range(num_features):
    mean, std = train_data[f'feature_{i}'].mean(), train_data[f'feature_{i}'].std()
    train_data[f'feature_{i}'] = train_data[f'feature_{i}'].apply(lambda x : (x-mean)/std)
    test_data[f'feature_{i}'] = test_data[f'feature_{i}'].apply(lambda x : (x-mean)/std)

In [ ]:
label_dict = {val:idx for idx, val in enumerate(sorted(train_data['target'].unique()))}
train_data['target'] = train_data['target'].map(label_dict)

target = train_data['target']
train_data.drop(['target'], inplace=True, axis=1)

In [ ]:
train_data = train_data.values
target = target.values
target =  to_categorical(target)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_data,target, test_size=0.18, random_state=220, stratify=target)

In [ ]:
num_classes = 4

In [ ]:
def dnn_model():
    input_layer =Input(shape=(num_features,))
    #layer_1
    dense_1=  Dense(512, activation='relu')(input_layer)
    batch_norm_1 = layers.BatchNormalization()(dense_1)
    
    dense_2 = Dense(256, activation='relu')(batch_norm_1)
    batch_norm_2 = layers.BatchNormalization()(dense_2)
    drop_1= layers.Dropout(0.2)(batch_norm_2)
    
    dense_3 = Dense(256, activation='relu')(drop_1)
    batch_norm_3 = layers.BatchNormalization()(dense_3)
    drop_2 = layers.Dropout(0.2)(batch_norm_3)
    
    
    dense_4 = Dense(128, activation='relu')(drop_2)
    batch_norm_4 = layers.BatchNormalization()(dense_4)
    drop_3 = layers.Dropout(0.2)(batch_norm_4)
    
    
    #layer_2
    dense_1_2=  Dense(1024, activation='relu')(input_layer)
    batch_norm_1_2 = layers.BatchNormalization()(dense_1_2)
    
    dense_2_2 = Dense(512, activation='relu')(batch_norm_1_2)
    batch_norm_2_2 = layers.BatchNormalization()(dense_2_2)
    drop_1_2= layers.Dropout(0.2)(batch_norm_2_2)
    
    dense_3_2 = Dense(512, activation='relu')(drop_1_2)
    batch_norm_3_2 = layers.BatchNormalization()(dense_3_2)
    drop_2_2 = layers.Dropout(0.2)(batch_norm_3_2)
    
    
    dense_4_2 = Dense(256, activation='relu')(drop_2_2)
    batch_norm_4_2 = layers.BatchNormalization()(dense_4_2)
    drop_3_2 = layers.Dropout(0.2)(batch_norm_4_2)
    
    
    dense_5_2 = Dense(256, activation='relu')(drop_3_2)
    batch_norm_5_2 = layers.BatchNormalization()(dense_5_2)
    drop_4_2 = layers.Dropout(0.2)(batch_norm_5_2)
    
    
    dense_6_2 = Dense(256, activation='relu')(drop_4_2)
    batch_norm_6_2 = layers.BatchNormalization()(dense_6_2)
    drop_5_2 = layers.Dropout(0.2)(batch_norm_6_2)
    
    
    concat = layers.concatenate([drop_3, drop_5_2, input_layer])
    
    
    x= Dense(64, activation='relu')(concat)
    y= Dense(128, activation='relu')(concat)
    
    batchnorm_x1=layers.BatchNormalization()(x)
    drop_x1 = layers.Dropout(0.3)(batchnorm_x1)
    
    dense_x2 = Dense(32, activation='relu')(drop_x1)
    batchnorm_x2=layers.BatchNormalization()(dense_x2)
    drop_x2 = layers.Dropout(0.3)(batchnorm_x2)
    
    
    dense_x3 = Dense(32, activation='relu')(drop_x2)
    batchnorm_x3=layers.BatchNormalization()(dense_x3)
    drop_x3 = layers.Dropout(0.2)(batchnorm_x3)
    
    
    
    
    batchnorm_y1=layers.BatchNormalization()(y)
    drop_y1 = layers.Dropout(0.3)(batchnorm_y1)
    
    dense_y2 = Dense(128, activation='relu')(drop_y1)
    batchnorm_y2=layers.BatchNormalization()(dense_y2)
    drop_y2 = layers.Dropout(0.3)(batchnorm_y2)
    
    
    dense_y3 = Dense(64, activation='relu')(drop_y2)
    batchnorm_y3=layers.BatchNormalization()(dense_y3)
    drop_y3 = layers.Dropout(0.2)(batchnorm_y3)
    
    
    
    concat_2 = layers.concatenate([drop_x3, drop_y3, input_layer])
    
    
    dense_final = Dense(16, activation= 'relu')(concat_2)
    batch = layers.BatchNormalization()(dense_final)
    
    final = Dense(num_classes, activation='softmax')(batch)
    
    model = Model(inputs = input_layer, outputs=final)
    return model
    
    
    
    
    
    
    
    
    
    
    
    

In [ ]:
model = dnn_model()
model.summary()

In [ ]:
plot_model(model, to_file='model.png', show_shapes=True, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
)

In [ ]:
model.compile(loss=['categorical_crossentropy','mse'], optimizer='adam', metrics='accuracy')

In [ ]:
history=  model.fit(x_train, y_train, batch_size=500, epochs=30, verbose =1, validation_data = (x_test, y_test))

In [ ]:


fig, ax = plt.subplots(figsize=(20,8))
sns.lineplot(x = history.epoch, y = history.history['loss'])
sns.lineplot(x = history.epoch, y = history.history['val_loss'])
ax.set_title('Learning Curve (Loss)')
ax.set_ylabel('Loss')
ax.set_xlabel('Epoch')
ax.legend(['train', 'test'], loc='best')
plt.show()



In [ ]:
predictions = pd.DataFrame(model.predict(test_data))
predictions.columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4']
predictions['id'] = test['id']
predictions = predictions[['id', 'Class_1', 'Class_2', 'Class_3', 'Class_4']]

predictions.to_csv("/kaggle/working/TSP_May_sub.csv", index=False)
predictions